# Twitter Pre-processing

Twitter Pre-processing to build following:

1. Twitter Timeline Data
2. Twitter User Profile Data
3. Twitter Most Frequent Words Data
4. Twitter Most Frequent Hashtags Data
5. Twitter Most Frequent Mentions Data

In [1]:
import json
import requests
from requests_oauthlib import OAuth1
import json
from urllib.parse import urlparse
import matplotlib.pyplot as plt
import tweepy
import seaborn as sns
%matplotlib inline
import os
import pandas as pd
import seaborn as sns
from pathlib import Path

plt.style.use('seaborn')

In [2]:
campus = "iu"
twitter_file = "iu"

## Settings

In [3]:
raw_data_dir = f"../../data/raw_data/{campus}/twitter/"
raw_file_data = f"{twitter_file}_twitter_data.csv"
raw_file_userinfo = f"{twitter_file}_twitter_user_profile_data.csv"


processed_data_dir =  f"../../data/processed_data/{campus}/twitter/"
file = f"{twitter_file}_twitter_data.xlsx"


####

### twitter data - main sheet
twitter_data_sheetname = "twitter_data"

## user profile sheet 
twitter_user_profile_data_sheetname = "user_profile_data"

## Most frequent words
most_frequent_words_sheetname = "most_freq_100_words"

## Most frequent hashtags
most_frequent_hashtags_sheetname = "most_freq_100_hashtags"

## Most frequent screennames
most_frequent_screennames_sheetname = "most_freq_100_mentions"

###



In [4]:
import nltk
stop_words =   nltk.corpus.stopwords.words('english') + [
 '.',
 ',',
 '--',
 '\'s',
 '?',
 ')',
 '(',
 ':',
 '\'',
 '\'re',
 '"',
 '-',
 '}',
 '{',
 u'—',
     
 ]

In [5]:
## Create Directory if it doesnt exists
def make_dir_if_not_exists(directory):
    dir_path = Path(directory)
    if not dir_path.exists():
        os.makedirs(dir_path)
    else:
        print("Path exists")
    
make_dir_if_not_exists(processed_data_dir)
    

Path exists


## Read Data

In [6]:
import shutil  

## raw file
#shutil.copyfile(os.path.join(raw_data_dir,raw_file_data), os.path.join(processed_data_dir,raw_file_data))
#print("Copied data file!")
#shutil.copyfile(os.path.join(raw_data_dir,raw_file_userinfo), os.path.join(processed_data_dir,raw_file_userinfo))
#print("Copied user file!")



In [7]:
## read data
import csv
df = pd.read_csv(os.path.join(raw_data_dir,raw_file_data))
user_data_df = pd.read_csv(os.path.join(raw_data_dir,raw_file_userinfo))

## save currentfile 
#df.to_excel(os.path.join(processed_data_dir,"indiana-university-bl_twitter_data.xls"),index=False)

df["created"] = pd.to_datetime(df["created"])

In [8]:
print("No of rows and columns: {}, min date: {}, max date: {}".format(df.shape,df['created'].min(),df['created'].max()))
      

No of rows and columns: (2619, 22), min date: 2018-07-01 22:26:31, max date: 2019-06-28 21:05:01


In [9]:
df.head()

,created,text,sentiment,tweet_id,truncated,user,user_followers_count,user_friends_count,user_favourites_count,user_statuses_count,...,favorite_count,reply_count,at_mentions,hashtags,urls,source,lat,long,tweet_type,words
0,2019-06-28 21:05:01,RT @IUbicentennial: GIVEAWAY TIME 🤩\n\nThere’s...,Positive,1144713335302230016,False,IndianaUniv,74668,450,5109,17988,...,0,0,IUbicentennial,NaN,NaN,"<a href=""https://sproutsocial.com"" rel=""nofoll...",NaN,NaN,retweet,giveaway time ’ day open bicentenni box want c...
1,2019-06-28 20:38:53,RT @GovHolcomb: Thank you to @IUBloomington Pr...,Neutral,1144706757543047169,False,IndianaUniv,74668,450,5109,17988,...,0,0,"GovHolcomb, IUBloomington",NaN,NaN,"<a href=""https://sproutsocial.com"" rel=""nofoll...",NaN,NaN,retweet,thank presid michael mcrobbi give tour tremend...
2,2019-06-28 13:49:38,3. MORE. DAYS. https://t.co/L1LFQFh3Mt,Neutral,1144603769054912512,False,IndianaUniv,74668,450,5109,17988,...,36,0,NaN,NaN,https://t.co/L1LFQFh3Mt,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,NaN,quote,day
3,2019-06-27 19:31:01,♦ 200 years of tradition.\n♦ 200 years of lega...,Neutral,1144327291427860481,True,IndianaUniv,74668,450,5109,17988,...,79,0,NaN,NaN,https://t.co/gn81xVvOJo,"<a href=""https://sproutsocial.com"" rel=""nofoll...",NaN,NaN,tweet,♦ 200 year tradit ♦ 200 year legaci ♦ 200 year...
4,2019-06-27 14:33:02,WOW. 🤩 \n\n20 programs across 7 IU campuses we...,Positive,1144252301693112321,True,IndianaUniv,74668,450,5109,17988,...,29,0,NaN,NaN,https://t.co/TfZBPI1HFq,"<a href=""https://sproutsocial.com"" rel=""nofoll...",NaN,NaN,tweet,wow program across campus award grant total 20...


In [10]:
pd.set_option('display.max_colwidth',-1)



## Most Frequent Words


In [11]:
from nltk.tokenize import TweetTokenizer
from nltk.stem.porter import PorterStemmer

def tweet_tokenizer(txt):
    try:
        tokenizer = TweetTokenizer()
        all_tokens = tokenizer.tokenize(txt.lower())
        # this line filters out all tokens that are entirely non-alphabetic characters
        filtered_tokens = [t for t in all_tokens if t.islower()]
        # filter out all tokens that are <=2 chars
        filtered_tokens = [x for x in filtered_tokens if len(x)>2]
    except IndexError:
        filtered_tokens = []
    return(filtered_tokens)


tweets_text = str(df["text"])

In [12]:
tweet_tokenizer(tweets_text)

['@iubicentennial',
 'giveaway',
 'time',
 'nthere',
 'only',
 'more',
 'days',
 'until',
 'you',
 'can',
 'open',
 'our',
 'bicentennial',
 'boxes',
 'want',
 'your',
 'chance',
 'win',
 'one',
 '@govholcomb',
 'thank',
 'you',
 '@iubloomington',
 'president',
 'michael',
 'mcrobbie',
 'for',
 'giving',
 'tour',
 'the',
 'tremendous',
 'luddy',
 'hall',
 'https://t.co/w8l6…',
 'more',
 'days',
 'https://t.co/l1lfqfh3mt',
 'years',
 'tradition',
 'years',
 'legacies',
 'years',
 'memories',
 "nit's",
 'time',
 'unbox',
 'those',
 'years',
 'and',
 'https://t.co/gn81xvvojo',
 'wow',
 'n20',
 'programs',
 'across',
 'campuses',
 'were',
 'awarded',
 'grants',
 'totaling',
 'from',
 'the',
 "women's",
 'philanthropy',
 'https://t.co/tfzbpi1hfq',
 '@iubicentennial',
 'giveaway',
 'time',
 'want',
 'win',
 'some',
 'bicentennial',
 'swag',
 'help',
 'kick',
 'off',
 'your',
 'bicentennial',
 'celebrations',
 'tweet',
 'pictures',
 '@iue_entr',
 'preparing',
 'for',
 '@iubicentennial',
 'ref

In [13]:
## Get most frequent words
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')

def get_most_frequent_words(text_series, stop_words = None, ngram_range = (1,2)):
    
    count_vectorizer = CountVectorizer(analyzer = "word",
                                       tokenizer = tweet_tokenizer,
                                       stop_words = stop_words,
                                       ngram_range = ngram_range 
                                      )
    
    term_freq_matrix = count_vectorizer.fit_transform(text_series)
        
    terms = count_vectorizer.get_feature_names()
    term_frequencies = term_freq_matrix.sum(axis = 0).tolist()[0]
    
    
    term_freq_df = (pd.DataFrame(list(zip(terms, term_frequencies)), columns = ["token","count"])
                    .set_index("token")
                    .sort_values("count",ascending = False))
    
    ### Avoiding hashtags and mentions
    data = term_freq_df.index.str.startswith(("@","#"))

    data = [not(data[i]) for i in range(len(data))]
    term_freq_df = term_freq_df[data]
    return term_freq_df

term_freq_df = get_most_frequent_words(df["text"],stop_words = stop_words)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mchivuku/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
term_freq_df = term_freq_df[:100]

## Most Frequent Hashtags and Screennames

In [15]:
df.columns

Index(['created', 'text', 'sentiment', 'tweet_id', 'truncated', 'user',
       'user_followers_count', 'user_friends_count', 'user_favourites_count',
       'user_statuses_count', 'retweeted_status', 'retweet_count',
       'favorite_count', 'reply_count', 'at_mentions', 'hashtags', 'urls',
       'source', 'lat', 'long', 'tweet_type', 'words'],
      dtype='object')

In [16]:
hashtags = []
screen_names = []
for index, row in df.iterrows():
    if isinstance(row["hashtags"], str):
        hashtags.extend([ t for t in str(row["hashtags"]).split(", ")])
    if isinstance(row["at_mentions"], str):
        screen_names.extend([t for t in str(row["at_mentions"]).split(", ")]) 
    


In [17]:
print("Hashtags: ", len(hashtags),"Screen_names:", len(screen_names))


Hashtags:  1503 Screen_names: 3199


#### Build Hashtag and Mentions table

In [18]:
from prettytable import PrettyTable
from collections import Counter
for label, data in (('Screen Name', screen_names),
                    ('Hashtag', hashtags)):
    pt = PrettyTable(field_names=[label, 'Count'])
    
    
    ### Merge duplicates
    c = {}
   
    for k in list(data):
        keys = [str.lower(p) for p in c.keys()]
        if str.lower(k) in keys:
            current_keys = list(c.keys())
            match_index = keys.index(str.lower(k))
            match = current_keys[match_index]
            c[match] += 1
        else:
            c[k] = 1
    
    if label == 'Screen Name':
        screenname_df = pd.DataFrame.from_dict(c, orient='index').reset_index()
        screenname_df = screenname_df.rename(columns={'index':'screen_name', 0:'count'}).sort_values("count",ascending = False)
        print(screenname_df.head())
    else:
        hashtag_df = pd.DataFrame.from_dict(c, orient='index').reset_index()
        hashtag_df = hashtag_df.rename(columns= {'index':"hashtag",0:'count'}).sort_values("count",ascending = False)
        print(hashtag_df.head())
    

      screen_name  count
5   IndianaUniv    304  
2   IUBloomington  279  
17  IUPUI          190  
4   IUFoundation   80   
19  IUNewsroom     65   
   hashtag  count
36  IUDay   617  
9   IUBOT   147  
37  myiuk   50   
25  iueast  36   
55  IU      25   


In [19]:
hashtag_df.head()

,hashtag,count
36,IUDay,617
9,IUBOT,147
37,myiuk,50
25,iueast,36
55,IU,25


In [20]:
print(f"Screennames: {screenname_df.shape}, hashtags: {hashtag_df.shape}")

Screennames: (850, 2), hashtags: (362, 2)


### Top 100 - mentions and hashtags

In [21]:
screenname_df, hashtag_df  = screenname_df[:100], hashtag_df[:100]

### Add Created Date


In [22]:
df['created date'] = df["created"].dt.date


## Build Excel Sheets

In [23]:
sentiment_df = df.loc[:,['sentiment']]


In [24]:
#df = df.drop(['Created Date'],axis=1)
df.head()

,created,text,sentiment,tweet_id,truncated,user,user_followers_count,user_friends_count,user_favourites_count,user_statuses_count,...,reply_count,at_mentions,hashtags,urls,source,lat,long,tweet_type,words,created date
0,2019-06-28 21:05:01,RT @IUbicentennial: GIVEAWAY TIME 🤩\n\nThere’s only 3 more days until you can open our IU Bicentennial boxes. Want your chance to win one? Tw…,Positive,1144713335302230016,False,IndianaUniv,74668,450,5109,17988,...,0,IUbicentennial,NaN,NaN,"<a href=""https://sproutsocial.com"" rel=""nofollow"">Sprout Social</a>",NaN,NaN,retweet,giveaway time ’ day open bicentenni box want chanc win one …,2019-06-28
1,2019-06-28 20:38:53,RT @GovHolcomb: Thank you to @IUBloomington President Michael McRobbie for giving me a tour of the tremendous Luddy Hall! https://t.co/W8l6…,Neutral,1144706757543047169,False,IndianaUniv,74668,450,5109,17988,...,0,"GovHolcomb, IUBloomington",NaN,NaN,"<a href=""https://sproutsocial.com"" rel=""nofollow"">Sprout Social</a>",NaN,NaN,retweet,thank presid michael mcrobbi give tour tremend luddi hall,2019-06-28
2,2019-06-28 13:49:38,3. MORE. DAYS. https://t.co/L1LFQFh3Mt,Neutral,1144603769054912512,False,IndianaUniv,74668,450,5109,17988,...,0,NaN,NaN,https://t.co/L1LFQFh3Mt,"<a href=""https://mobile.twitter.com"" rel=""nofollow"">Twitter Web App</a>",NaN,NaN,quote,day,2019-06-28
3,2019-06-27 19:31:01,♦ 200 years of tradition.\n♦ 200 years of legacies.\n♦ 200 years of memories.\n\nIt's time to unbox those 200 years and… https://t.co/gn81xVvOJo,Neutral,1144327291427860481,True,IndianaUniv,74668,450,5109,17988,...,0,NaN,NaN,https://t.co/gn81xVvOJo,"<a href=""https://sproutsocial.com"" rel=""nofollow"">Sprout Social</a>",NaN,NaN,tweet,♦ 200 year tradit ♦ 200 year legaci ♦ 200 year memori time unbox 200 year …,2019-06-27
4,2019-06-27 14:33:02,"WOW. 🤩 \n\n20 programs across 7 IU campuses were awarded grants totaling $200,000 from the IU Women's Philanthropy Le… https://t.co/TfZBPI1HFq",Positive,1144252301693112321,True,IndianaUniv,74668,450,5109,17988,...,0,NaN,NaN,https://t.co/TfZBPI1HFq,"<a href=""https://sproutsocial.com"" rel=""nofollow"">Sprout Social</a>",NaN,NaN,tweet,"wow program across campus award grant total 200,000 women philanthropi …",2019-06-27


<ul>
<li> Twitter Data </li>
<li> User Profile Data </li>
<li> Most Frequent Words </li>
<li> Most Frequent Hashtags </li>
<li> Most Frequent Mentions </li>

</ul>

In [25]:
user_data_df

,user_id,name,screen_name,followers_count,friends_count,favourites_count,statuses_count,total_reach,avg_follower_cnt
0,100367386,Indiana University,IndianaUniv,74668,450,5109,17988,10968413,146.9


In [26]:

with pd.ExcelWriter(os.path.join(processed_data_dir,file),engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name=twitter_data_sheetname,index=False)
    user_data_df.to_excel(writer, sheet_name=twitter_user_profile_data_sheetname,index=False)
    term_freq_df.to_excel(writer, sheet_name=most_frequent_words_sheetname)
    screenname_df.to_excel(writer, sheet_name=most_frequent_screennames_sheetname,index=False)
    hashtag_df.to_excel(writer, sheet_name=most_frequent_hashtags_sheetname,index=False)
    writer.save()
    